## setup

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from huggingface_hub import login
from dotenv import load_dotenv
from datasets import load_dataset
import os
import wandb
from peft import LoraConfig, get_peft_model
from pipeline.main import run_eval
import json

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
load_dotenv()
login(token = os.environ['HF_TOKEN'])
wandb.login()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: atharva_nihalani (atharva_nihalani-brown-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## FT Dataset

### Load Model / Dataset

In [ ]:
'''
model_name = 'meta-llama/Llama-3.1-8B-Instruct'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=t.float16,
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, pad_side="left")
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    quantization_config=bnb_config,
)
tokenizer.pad_token = tokenizer.eos_token'''

# model_name = 'unsloth/Meta-Llama-3.1-8B-Instruct'
model_name = 'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit'


tokenizer = AutoTokenizer.from_pretrained(model_name, pad_side="left")
tokenizer.pad_token = tokenizer.eos_token


model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    torch_dtype="auto",
)

In [ ]:
with open("/root/srf-project/data/codenet_questions/filtered_problem_descriptions.json", "r") as f:
    problem_descriptions = json.load(f)

def get_filtered_dataset(lang):
    dataset = load_dataset('iNeil77/CodeNet', lang, split='train')
    dataset = dataset.select_columns(['p_id', 'language', 'status', 'code'])
    dataset = dataset.filter(lambda x: x['status']=='Accepted')
    dataset = dataset.filter(lambda x: x['p_id'] in problem_descriptions.keys())

    return dataset

def get_train_test(dataset, train_size=10000, test_size=500):
    shuffled = dataset.shuffle(seed=47)
    train_set = shuffled.select(range(train_size))
    test_set = shuffled.select(range(train_size, train_size + test_size))

    return train_set, test_set

filtered = get_filtered_dataset('Java')
train_set, test_set = get_train_test(filtered)

In [ ]:
def tokenize(record):
    code = record['code']
    tokens = tokenizer(
        code, 
        truncation=True,
        max_length=1024,
    )

    return tokens

train_set = train_set.map(tokenize, batched=True, num_proc=32)
train_set = train_set.select_columns(['input_ids', 'attention_mask'])

test_set = test_set.map(tokenize, batched=True, num_proc=32)
test_set = test_set.select_columns(['input_ids', 'attention_mask'])


In [ ]:
# for i in range(10):
#     idx = random.randint(0, dataset_cpp.num_rows)
#     code = dataset_cpp[idx]['code']
#     print(code)
#     print('--x---x---x--\n')

In [ ]:
# api = wandb.Api()
# run = api.run("atharva_nihalani-brown-university/huggingface/diw9fexc")
# metrics_dataframe = run.history()
# # metrics_dataframe.to_csv("metrics.csv")

In [ ]:
# # Access the GPU metrics
# gpu_memory_util = metrics_dataframe.get("gpu.0.memory")  # For the first GPU
# gpu_memory_alloc = metrics_dataframe.get("gpu.0.memoryAllocated")
# print(gpu_memory_util)
# print(gpu_memory_alloc)

In [ ]:
# print(metrics_dataframe.columns)

### FT Configs

In [ ]:
configs = [
    {'r': 4,  'alpha': 16, 'dropout': 0.00, 'accum_steps': 1, 'lr': 1e-4},   # A
    {'r': 4,  'alpha': 32, 'dropout': 0.10, 'accum_steps': 2, 'lr': 2e-4},   # B
    {'r': 8,  'alpha': 32, 'dropout': 0.05, 'accum_steps': 4, 'lr': 1e-4},   # C
    {'r': 16, 'alpha': 64, 'dropout': 0.00, 'accum_steps': 1, 'lr': 5e-4},   # D
    {'r': 16, 'alpha': 32, 'dropout': 0.10, 'accum_steps': 2, 'lr': 2e-4},   # E
    {'r': 8,  'alpha': 64, 'dropout': 0.05, 'accum_steps': 1, 'lr': 5e-5},   # F
]

### FT #A

In [ ]:
lora_config = LoraConfig(
    r=configs[0]['r'],
    lora_alpha=configs[0]['alpha'],
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=configs[0]['dropout'],
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./llama3-java-finetune",
    eval_strategy='steps',
    eval_steps=0.1,
    eval_on_start=True,
    # per_device_train_batch_size=8, 
    auto_find_batch_size=True,
    gradient_accumulation_steps=configs[0]['accum_steps'],
    dataloader_num_workers=16,
    dataloader_persistent_workers=True,
    learning_rate=configs[0]['lr'],
    num_train_epochs=1,  
    bf16=True,
    save_steps=0.2,
    save_total_limit=3,
    logging_steps=0.02,
    report_to="wandb",
    logging_first_step=True,
    run_name='quantized',
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # For causal LM
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
wandb.finish()

### FT #B

In [ ]:
lora_config = LoraConfig(
    r=configs[1]['r'],
    lora_alpha=configs[1]['alpha'],
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=configs[1]['dropout'],
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./llama3-java-finetune",
    eval_strategy='steps',
    eval_steps=0.1,
    eval_on_start=True,
    # per_device_train_batch_size=8, 
    auto_find_batch_size=True,
    gradient_accumulation_steps=configs[1]['accum_steps'],
    dataloader_num_workers=16,
    dataloader_persistent_workers=True,
    learning_rate=configs[1]['lr'],
    num_train_epochs=1,  
    bf16=True,
    save_steps=0.2,
    save_total_limit=3,
    logging_steps=0.02,
    report_to="wandb",
    logging_first_step=True,
    run_name='quantized',
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # For causal LM
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
wandb.finish()

### FT #C

In [ ]:
lora_config = LoraConfig(
    r=configs[2]['r'],
    lora_alpha=configs[2]['alpha'],
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=configs[2]['dropout'],
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./llama3-java-finetune",
    eval_strategy='steps',
    eval_steps=0.1,
    eval_on_start=True,
    # per_device_train_batch_size=8, 
    auto_find_batch_size=True,
    gradient_accumulation_steps=configs[2]['accum_steps'],
    dataloader_num_workers=16,
    dataloader_persistent_workers=True,
    learning_rate=configs[2]['lr'],
    num_train_epochs=1,  
    bf16=True,
    save_steps=0.2,
    save_total_limit=3,
    logging_steps=0.02,
    report_to="wandb",
    logging_first_step=True,
    run_name='quantized',
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # For causal LM
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
wandb.finish()

### RunEval

In [ ]:
args = {
    'model': 'hf/local',
    'model_path': '/root/srf-project/test_dir',
    'device': 'auto',
    'torch_dtype': 'auto'
}

run_eval('java', args, samples=164)

In [ ]:
def get_dataset_new(lang):
    dataset = load_dataset('iNeil77/CodeNet', lang, split='train')
    dataset = dataset.select_columns(['p_id', 'language', 'status', 'code'])
    dataset = dataset.filter(lambda x: x['status']=='Accepted')
    shuffled = dataset.shuffle(seed=47).select(range(10500))

    return shuffled

In [ ]:
dataset = get_dataset_new('Java')

In [ ]:
out = dataset[300]['code']
print(out)

### Trial Sweep

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./llama3-java-finetune",
    eval_strategy='steps',
    eval_steps=0.1,
    eval_on_start=True,
    per_device_train_batch_size=8, 
    gradient_accumulation_steps=4,
    dataloader_num_workers=16,
    dataloader_persistent_workers=True,
    learning_rate=2e-4,
    num_train_epochs=1,  
    bf16=True,
    save_steps=0.2,
    save_total_limit=3,
    logging_steps=0.02,
    report_to="wandb",
    logging_first_step=True,
    run_name='temp_run',
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # For causal LM
)

In [ ]:
def model_init(trial):
    return AutoModelForCausalLM.from_pretrained(
        model_name, 
        device_map="auto", 
        torch_dtype="auto",
    )


trainer = Trainer(
    # model=model,
    model_init=model_init,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    data_collator=data_collator,
)

In [ ]:
def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "objective", "goal": "minimize"},
        "parameters": {
            "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 1e-4},
            "per_device_train_batch_size": {"values": [4, 8]},
            "gradient_accumulation_steps": {"values": [1, 2, 4, 8]},
            # "r": {"values": [2, 4, 8, 16]},
            # "lora_alpha": {"values": [16, 32, 64, 128]},
            # "lora_dropout": {"min": 0.0, "max": 0.2}, 
        },
    }

In [ ]:
best_trials = trainer.hyperparameter_search( 
    direction="minimize",
    backend="wandb",
    hp_space=wandb_hp_space,
    n_trials=4,
    # compute_objective=compute_objective,
)

### Misc

In [ ]:
trainer.train()

In [ ]:
wandb.finish()

In [ ]:
args = {
    'model': 'hf/local',
    'model_path': '/root/srf-project/llama3-java-finetune/checkpoint-1250',
    'device': 'auto',
    'torch_dtype': 'auto'
}

run_eval('java', model_args=args, epochs=3)

### GPU Deets

In [ ]:
import torch as t
import gc

free_memory, total_memory = t.cuda.mem_get_info()

# Convert bytes to GB
free_memory_gb = free_memory / (1024 * 1024 * 1024)
total_memory_gb = total_memory / (1024 * 1024 * 1024)
mem_used = t.cuda.device_memory_used() / (1024 ** 3)

print(f"Free GPU Memory: {free_memory_gb:.2f} GB")
print(f"Total GPU Memory: {total_memory_gb:.2f} GB")
print(f'Memory Used: {mem_used:.2f} GB')

In [ ]:
print(t.cuda.memory_allocated() / 1024**2, "MB allocated")
print(t.cuda.memory_reserved() / 1024**2, "MB reserved")

In [ ]:
print(t.cuda.memory_allocated() / 1024**2, "MB allocated")
print(t.cuda.memory_reserved() / 1024**2, "MB reserved")